# Readme
This is a demo notebook that shows how the core schema is loaded and how to add data

If you have not installed `djimaging`, please go to project directory and install it via `pip install --editable .`.

In [ ]:
%load_ext autoreload
%autoreload 2

# Data Base

## Prepare dj config

In [ ]:
import os
import datajoint as dj

In [ ]:
username = !whoami
username = username[0]
username

In [ ]:
home_directory = os.path.expanduser("~")
home_directory

In [ ]:
# Set config file
config_file = f'{home_directory}/datajoint/dj_{username}_conf.json'
assert os.path.isfile(config_file), f'Set the path to your config file: {config_file}'

In [ ]:
# Define a schema name or use the default name for your personal test schema
# It should start with ageuler and have some meaningful name after that
schema_name = f"ageuler_{username}_test"

In [ ]:
# Load configuration for user
dj.config.load(config_file)
dj.config['schema_name'] = schema_name
print("schema_name:", dj.config['schema_name'])
dj.conn()

## Create or load schema

In [ ]:
from djimaging.schemas.advanced_schema import *

In [ ]:
from djimaging.utils.dj_utils import activate_schema

activate_schema(schema=schema, create_schema=True, create_tables=True)
schema

In [ ]:
# Draw the ERD of your schema, this may take a while
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

dj.ERD(schema)

## Upload user

In [ ]:
userinfo = {
    'experimenter': 'DataJointTestData', # Replace this if you want to use your own data
    'data_dir': '/gpfs01/euler/data/Data/DataJointTestData/', # Replace this if you want to use your own data
    'datatype_loc': 0,
    'animal_loc': 1,
    'region_loc': 2,
    'field_loc': 3,
    'stimulus_loc': 4,
    'condition_loc': 5,
}

assert os.path.isdir(userinfo['data_dir'])

In [ ]:
UserInfo().upload_user(userinfo)
UserInfo()

In [ ]:
# Plot the data files in the selected folder
UserInfo().plot1(key=None, show_pre=False, show_raw=False, show_header=True)

## Populate data

### Experiments

In [ ]:
Experiment().rescan_filesystem(verboselvl=2)
Experiment()

### Fields

In [ ]:
Field()
Field().rescan_filesystem(verboselvl=2)

In [ ]:
# If you call plot1 on a djimaging table, it will plot the given key.
# If you pass key=None or no key it will pick a key at random.
# This is implemented for most core tables and can be useful to get a quick impression of the data.
Field().plot1(key=None)

#### Rois

In [ ]:
Roi().populate()
Roi()

In [ ]:
Roi().plot1(key=None)

### Stimuli

#### Add default stimuli

In [ ]:
Stimulus()

In [ ]:
# To compute receptive fields, the noise stimulus trace must be loaded and set!
import h5py

with h5py.File("/gpfs01/euler/data/Resources/Stimulus/noise.h5", "r") as f:
    noise_stimulus = f['stimulusarray'][:].T.astype(int)

In [ ]:
Stimulus().add_nostim(skip_duplicates=True)
Stimulus().add_chirp(spatialextent=1000, stim_name='gChirp', alias="chirp_gchirp_globalchirp", skip_duplicates=True)
Stimulus().add_chirp(spatialextent=300, stim_name='lChirp', alias="lchirp_localchirp", skip_duplicates=True)
Stimulus().add_noise(stim_name='noise', pix_n_x=20, pix_n_y=15, pix_scale_x_um=30, pix_scale_y_um=30, stim_trace=noise_stimulus, skip_duplicates=True)
Stimulus().add_movingbar(skip_duplicates=True)

In [ ]:
# Stimulus().add_stimulus(stim_name='my_stimulus', alias="stimname_stimnameabbr_alternativename", isrepeated=True, ntrigger_rep=6,
#                         trial_info=[1, 2, 3, 4, 5, 6], skip_duplicates=True)

In [ ]:
Stimulus()

#### Presentations

In [ ]:
Presentation().populate()
Presentation()

In [ ]:
Presentation().plot1(key=None)

### Traces

In [ ]:
Traces().populate()
Traces()

In [ ]:
PreprocessParams().add_default(skip_duplicates=True)
PreprocessParams()

In [ ]:
PreprocessTraces().populate(display_progress=True)
PreprocessTraces()

In [ ]:
PreprocessTraces().plot1(key=None)

In [ ]:
Snippets().populate(display_progress=True)
Snippets()

In [ ]:
Snippets().plot1(key=None)

In [ ]:
Averages().populate(display_progress=True)
Averages()

In [ ]:
Averages().plot({"stim_name": "gChirp"})

## Recording location

In [ ]:
OpticDisk().populate()
OpticDisk()

In [ ]:
RelativeFieldLocation().populate()
RelativeFieldLocation()

In [ ]:
RelativeFieldLocation().plot()

In [ ]:
RetinalFieldLocation().populate()
RetinalFieldLocation()

In [ ]:
RetinalFieldLocation().plot()